In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.13.0


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...


C:\Users\knh11\anaconda3\envs\tfpy3.10\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch: [ 0] [    0/  468] time: 1.4470, train_loss: 2.02565575, train_accuracy: 0.2109, test_Accuracy: 0.0974
Epoch: [ 0] [    1/  468] time: 1.9431, train_loss: 2.07795787, train_accuracy: 0.3203, test_Accuracy: 0.2962
Epoch: [ 0] [    2/  468] time: 2.5753, train_loss: 1.91482949, train_accuracy: 0.4766, test_Accuracy: 0.4113
Epoch: [ 0] [    3/  468] time: 3.4019, train_loss: 1.78721952, train_accuracy: 0.6953, test_Accuracy: 0.5898
Epoch: [ 0] [    4/  468] time: 4.1895, train_loss: 1.56178188, train_accuracy: 0.6328, test_Accuracy: 0.6143
Epoch: [ 0] [    5/  468] time: 4.9487, train_loss: 1.31995142, train_accuracy: 0.6641, test_Accuracy: 0.6522
Epoch: [ 0] [    6/  468] time: 5.6706, train_loss: 0.93636286, train_accuracy: 0.7969, test_Accuracy: 0.7284
Epoch: [ 0] [    7/  468] time: 6.3775, train_loss: 0.84462732, train_accuracy: 0.8438, test_Accuracy: 0.7961
Epoch: [ 0] [    8/  468] time: 6.9305, train_loss: 0.60917795, train_accuracy: 0.8359, test_Accuracy: 0.7574
Epoch: [ 0

Epoch: [ 0] [   74/  468] time: 55.7198, train_loss: 0.30504709, train_accuracy: 0.8984, test_Accuracy: 0.9210
Epoch: [ 0] [   75/  468] time: 56.4183, train_loss: 0.21355787, train_accuracy: 0.9531, test_Accuracy: 0.9285
Epoch: [ 0] [   76/  468] time: 57.1293, train_loss: 0.10685411, train_accuracy: 0.9688, test_Accuracy: 0.9332
Epoch: [ 0] [   77/  468] time: 57.7528, train_loss: 0.14567542, train_accuracy: 0.9453, test_Accuracy: 0.9349
Epoch: [ 0] [   78/  468] time: 58.5069, train_loss: 0.14789237, train_accuracy: 0.9531, test_Accuracy: 0.9326
Epoch: [ 0] [   79/  468] time: 59.2158, train_loss: 0.22119719, train_accuracy: 0.9531, test_Accuracy: 0.9281
Epoch: [ 0] [   80/  468] time: 59.8718, train_loss: 0.21385941, train_accuracy: 0.9609, test_Accuracy: 0.9299
Epoch: [ 0] [   81/  468] time: 60.5882, train_loss: 0.22274689, train_accuracy: 0.9453, test_Accuracy: 0.9330
Epoch: [ 0] [   82/  468] time: 61.2628, train_loss: 0.14479397, train_accuracy: 0.9609, test_Accuracy: 0.9373
E

Epoch: [ 0] [  148/  468] time: 107.0246, train_loss: 0.09492511, train_accuracy: 0.9766, test_Accuracy: 0.9472
Epoch: [ 0] [  149/  468] time: 107.8854, train_loss: 0.12001379, train_accuracy: 0.9531, test_Accuracy: 0.9439
Epoch: [ 0] [  150/  468] time: 108.7000, train_loss: 0.16532609, train_accuracy: 0.9531, test_Accuracy: 0.9438
Epoch: [ 0] [  151/  468] time: 109.3220, train_loss: 0.13981208, train_accuracy: 0.9688, test_Accuracy: 0.9447
Epoch: [ 0] [  152/  468] time: 109.9542, train_loss: 0.25377488, train_accuracy: 0.9297, test_Accuracy: 0.9466
Epoch: [ 0] [  153/  468] time: 110.5092, train_loss: 0.10328871, train_accuracy: 0.9844, test_Accuracy: 0.9453
Epoch: [ 0] [  154/  468] time: 111.3280, train_loss: 0.12596652, train_accuracy: 0.9609, test_Accuracy: 0.9432
Epoch: [ 0] [  155/  468] time: 112.0115, train_loss: 0.19103953, train_accuracy: 0.9531, test_Accuracy: 0.9443
Epoch: [ 0] [  156/  468] time: 112.7583, train_loss: 0.16020493, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 158.2361, train_loss: 0.11884399, train_accuracy: 0.9688, test_Accuracy: 0.9480
Epoch: [ 0] [  223/  468] time: 159.0538, train_loss: 0.15446359, train_accuracy: 0.9375, test_Accuracy: 0.9460
Epoch: [ 0] [  224/  468] time: 159.6177, train_loss: 0.18139225, train_accuracy: 0.9375, test_Accuracy: 0.9419
Epoch: [ 0] [  225/  468] time: 160.3571, train_loss: 0.13527538, train_accuracy: 0.9766, test_Accuracy: 0.9380
Epoch: [ 0] [  226/  468] time: 160.9681, train_loss: 0.13949390, train_accuracy: 0.9688, test_Accuracy: 0.9442
Epoch: [ 0] [  227/  468] time: 161.5327, train_loss: 0.11029041, train_accuracy: 0.9531, test_Accuracy: 0.9540
Epoch: [ 0] [  228/  468] time: 162.3480, train_loss: 0.12220927, train_accuracy: 0.9609, test_Accuracy: 0.9596
Epoch: [ 0] [  229/  468] time: 163.0223, train_loss: 0.14511609, train_accuracy: 0.9531, test_Accuracy: 0.9568
Epoch: [ 0] [  230/  468] time: 163.8518, train_loss: 0.16244769, train_accuracy: 0.9375, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 209.1974, train_loss: 0.07903767, train_accuracy: 0.9844, test_Accuracy: 0.9510
Epoch: [ 0] [  297/  468] time: 209.9303, train_loss: 0.10985468, train_accuracy: 0.9531, test_Accuracy: 0.9513
Epoch: [ 0] [  298/  468] time: 210.5157, train_loss: 0.09992760, train_accuracy: 0.9609, test_Accuracy: 0.9541
Epoch: [ 0] [  299/  468] time: 211.0643, train_loss: 0.07030036, train_accuracy: 0.9766, test_Accuracy: 0.9575
Epoch: [ 0] [  300/  468] time: 211.7657, train_loss: 0.09368938, train_accuracy: 0.9844, test_Accuracy: 0.9562
Epoch: [ 0] [  301/  468] time: 212.3911, train_loss: 0.09353468, train_accuracy: 0.9688, test_Accuracy: 0.9562
Epoch: [ 0] [  302/  468] time: 213.1038, train_loss: 0.15579818, train_accuracy: 0.9688, test_Accuracy: 0.9573
Epoch: [ 0] [  303/  468] time: 213.7311, train_loss: 0.11854807, train_accuracy: 0.9688, test_Accuracy: 0.9578
Epoch: [ 0] [  304/  468] time: 214.3424, train_loss: 0.09776329, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 259.4929, train_loss: 0.07800428, train_accuracy: 0.9766, test_Accuracy: 0.9633
Epoch: [ 0] [  371/  468] time: 260.1522, train_loss: 0.09103666, train_accuracy: 0.9766, test_Accuracy: 0.9636
Epoch: [ 0] [  372/  468] time: 260.7190, train_loss: 0.02228652, train_accuracy: 1.0000, test_Accuracy: 0.9619
Epoch: [ 0] [  373/  468] time: 261.2464, train_loss: 0.10732474, train_accuracy: 0.9688, test_Accuracy: 0.9600
Epoch: [ 0] [  374/  468] time: 261.9946, train_loss: 0.08713569, train_accuracy: 0.9766, test_Accuracy: 0.9621
Epoch: [ 0] [  375/  468] time: 262.7004, train_loss: 0.08668858, train_accuracy: 0.9766, test_Accuracy: 0.9652
Epoch: [ 0] [  376/  468] time: 263.3907, train_loss: 0.10987507, train_accuracy: 0.9609, test_Accuracy: 0.9663
Epoch: [ 0] [  377/  468] time: 264.0534, train_loss: 0.06247540, train_accuracy: 0.9844, test_Accuracy: 0.9661
Epoch: [ 0] [  378/  468] time: 264.7861, train_loss: 0.15281355, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 309.2962, train_loss: 0.07714126, train_accuracy: 0.9844, test_Accuracy: 0.9597
Epoch: [ 0] [  445/  468] time: 310.1435, train_loss: 0.18971199, train_accuracy: 0.9453, test_Accuracy: 0.9594
Epoch: [ 0] [  446/  468] time: 310.7561, train_loss: 0.08898298, train_accuracy: 0.9766, test_Accuracy: 0.9599
Epoch: [ 0] [  447/  468] time: 311.5513, train_loss: 0.08388090, train_accuracy: 0.9922, test_Accuracy: 0.9617
Epoch: [ 0] [  448/  468] time: 312.3584, train_loss: 0.08156151, train_accuracy: 0.9688, test_Accuracy: 0.9647
Epoch: [ 0] [  449/  468] time: 312.9159, train_loss: 0.06207072, train_accuracy: 0.9922, test_Accuracy: 0.9658
Epoch: [ 0] [  450/  468] time: 313.5204, train_loss: 0.12220608, train_accuracy: 0.9609, test_Accuracy: 0.9645
Epoch: [ 0] [  451/  468] time: 314.1665, train_loss: 0.06390639, train_accuracy: 0.9844, test_Accuracy: 0.9622
Epoch: [ 0] [  452/  468] time: 314.6549, train_loss: 0.04753748, train_accuracy: 0.9922, test_Accuracy:

# Test accuracy: 96.50%